In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend([WORKING_DIR_AND_PYTHON_PATHS])

Python 3.7.9 (default, Aug 31 2020, 07:22:35) 
[Clang 10.0.0 ] on darwin


NameError: name 'WORKING_DIR_AND_PYTHON_PATHS' is not defined